In [ ]:
!pip install python-barcode
!pip install check-digit-EAN13

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Conv2D, Add, ZeroPadding2D, UpSampling2D, Concatenate, MaxPooling2D, ZeroPadding2D, Lambda, Reshape, Flatten, Dense, concatenate
from tensorflow.compat.v1.image import resize_nearest_neighbor, resize_bilinear
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Lambda
from tensorflow.python.framework.ops import disable_eager_execution
from tensorflow.keras.callbacks import *
from functools import reduce
from PIL import Image
from matplotlib.colors import rgb_to_hsv, hsv_to_rgb
import cv2
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.utils import Sequence
import os 
import cv2
from barcode import EAN13, ISBN13
from barcode.writer import ImageWriter
from tqdm import tqdm
from check_digit_EAN13.check_digit import get_check_digit

In [ ]:
IMAGE_HEIGHT = 160
IMAGE_WIDTH = 320
BATCH_SIZE = 128
count = 15000
path = "dataset_2"

In [ ]:
if not os.path.exists(os.path.join(path, "with_numbers", "images")):
    os.makedirs(os.path.join(path, "with_numbers", "images"))
if not os.path.exists(os.path.join(path, "without_numbers", "images")):
    os.makedirs(os.path.join(path, "without_numbers", "images"))

In [ ]:
for i in tqdm(range(count)):
    random_code = np.random.randint(999999999999, 9999999999999, dtype=np.int64)
    random_code = get_check_digit(random_code)
    with open("{}/with_numbers/images/{}_{}_1.jpg".format(path, str(random_code), str(i)), "wb") as file:
        EAN13(str(random_code), writer=ImageWriter()).write(file)
    # if i % 2 == 0:
    image = cv2.imread("{}/with_numbers/images/{}_{}_1.jpg".format(path, str(random_code), str(i)), cv2.IMREAD_GRAYSCALE)
    image = np.asarray(image)[2:199, 63:460]
    image = cv2.resize(image, (320, 160))
    cv2.imwrite("{}/without_numbers/images/{}_{}_0.jpg".format(path, str(random_code), str(i)), image)

In [ ]:
class Datagenerator(Sequence):
    def __init__(self, image_set_path, batch_size):
        self.image_set_path = image_set_path
        self.batch_size = batch_size
        self.image_list = os.listdir(image_set_path)
        print(len(self.image_list))

    def __getitem__(self, index):
        batch_input_list = self.image_list[self.batch_size * index: self.batch_size * (1 + index)]
        
        X = np.zeros((self.batch_size, IMAGE_HEIGHT, IMAGE_WIDTH, 1))
        Y = np.zeros((self.batch_size, 13, 10))
        for i, image_name in enumerate(batch_input_list):
            image = cv2.imread(os.path.join(self.image_set_path, image_name), cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT))
            image = np.expand_dims(image, -1)
            image = image / 128
            X[i] = image
            Y[i] = self.encode_label(image_name.split("_")[0])            
        return X, Y

    def __len__(self):
        return len(self.image_list) // self.batch_size
    
    def encode_label(self, label):
        OHE = np.zeros((13, 10))
        for i, num in enumerate(label):
            num = int(num)
            OHE[i][num] = 1
        return OHE

In [ ]:
def custom_loss_tf(y_true, y_pred):
    y_true = tf.reshape(y_true, (-1, 13, 10))
    y_pred = tf.reshape(y_pred, (-1, 13, 10))
    soft_pred = tf.math.softmax(y_pred, axis=-1)
    cross_entropy = tf.losses.categorical_crossentropy(y_true=y_true, y_pred=soft_pred)
    sum_loss = tf.math.reduce_sum(cross_entropy)
    return sum_loss

In [ ]:
def Conv2D_block(inputs, out_channels, kernel_size=(1, 1), strides=(1, 1), padding="same"):
    conv = Conv2D(out_channels, kernel_size, strides=strides, padding=padding, use_bias=False)(inputs)
    bn = BatchNormalization()(conv)
    relu_out = ReLU()(bn)

    return relu_out

In [ ]:
def Bottleneck(out_channels, shortcut=True, group=1, e=0.5, inputs=None):
    if shortcut:
        c_ = int(out_channels * e)
        x1 = Conv2D_block(out_channels=c_, inputs=inputs)
        x2 = Conv2D_block(out_channels=out_channels, inputs=x1, kernel_size=(3, 3), strides=(1, 1))
        add = Add()([inputs, x2])
        add = ReLU()(add)
        return add
    else:
        c_ = int(out_channels * e)
        x1 = Conv2D_block(out_channels=c_, inputs=inputs)
        x2 = Conv2D_block(out_channels=out_channels, inputs=x1, kernel_size=(3, 3), strides=(1, 1))
        return x2

In [ ]:
def BottleneckCSP(out_channels, number=1, shortcut=True, group=1, e=0.5, inputs=None):
    c_ = int(out_channels * e)
    x1 = Conv2D_block(out_channels=c_, kernel_size=(1, 1), strides=(1, 1), inputs=inputs)

    for _ in range(number):
        x1 = Bottleneck(out_channels=c_, shortcut=shortcut, e=1.0, group=group, inputs=x1)
    y2 = Conv2D_block(out_channels=c_, kernel_size=(1, 1), strides=(1, 1), inputs=inputs)

    c = Concatenate()([x1, y2])
    c = Conv2D_block(out_channels=2 * c_, kernel_size=(1, 1), strides=(1, 1), inputs=c)
    return c

In [ ]:
def SPP(c1, c2, k=(5, 9, 13), inputs=None):
    c_ = c1 // 2  
    x = Conv2D_block(out_channels=c_, kernel_size=(1, 1), strides=(1, 1), inputs=inputs)
    x = Conv2D_block(out_channels=c2, kernel_size=(1, 1), strides=(1, 1), inputs=x)
    return x

In [ ]:
def Dense_layer(num_class, inputs=None):
    if len(inputs.shape) > 2:
        inputs = Flatten()(inputs)

    dense = Dense(num_class, activation="relu")(inputs)

    return dense

In [ ]:
def _create_model():
    input_layer = Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 1),
                            name="input")

    x0 = Conv2D_block(inputs=input_layer, out_channels=16, kernel_size=(3,3), strides=(2, 2))
    x1 = Conv2D_block(inputs=x0, out_channels=32, kernel_size=(3, 3), strides=(2, 2))
    x2 = BottleneckCSP(inputs=x1, number=1, out_channels=32, shortcut=True, group=1, e=0.5)
    x3 = Conv2D_block(inputs=x2, out_channels=64, kernel_size=(3, 3), strides=(2, 2))
    x4 = BottleneckCSP(inputs=x3, number=2, out_channels=64, shortcut=True, group=1, e=0.5)
    x5 = Conv2D_block(inputs=x4, out_channels=128, kernel_size=(3, 3), strides=(2, 2))
    x6 = BottleneckCSP(inputs=x5, number=3, out_channels=128, shortcut=True, group=1, e=0.5)    
    x7 = Conv2D_block(inputs=x6, out_channels=256, kernel_size=(3, 3), strides=(1, 1))
    x9 = BottleneckCSP(inputs=x7, number=1, out_channels=256, shortcut=True, group=1, e=0.5)
    x111 = SPP(64, 32, inputs=x9)

    # block 1
    x10 = Conv2D_block(inputs=x111, out_channels=32, kernel_size=(3, 3), strides=(2, 2))
    x10 = Dense_layer(130, x10)
    
    model = Model(input_layer, x10)
    
    return model

In [ ]:
model = _create_model()
model.summary()
model.compile(optimizer="adam", loss=custom_loss_tf, metrics=custom_loss_tf, run_eagerly=True)

In [ ]:
data_generator = Datagenerator("/kaggle/working/dataset_2/without_numbers/images", BATCH_SIZE) 
callbacks_ckpt = ModelCheckpoint("model_{epoch:03d}.h5", save_freq=10)
image_batch_shape = (BATCH_SIZE, IMAGE_HEIGHT, IMAGE_WIDTH, 1)
label_batch_shape = (BATCH_SIZE, 13, 10)
tf_train_generator = tf.data.Dataset.from_generator(lambda: map(tuple, data_generator), 
                                                    (tf.float32, tf.float32),
                                                    (image_batch_shape, label_batch_shape))
tf_train_generator = tf_train_generator.prefetch(buffer_size=tf.data.AUTOTUNE).cache()

In [ ]:
tf.config.run_functions_eagerly(True)
model.fit(tf_train_generator, epochs=500, callbacks=[callbacks_ckpt], verbose=1)
model.save("last_model.h5")

In [ ]:
path = "test"
if not os.path.exists(os.path.join("test", "with_numbers", "images")):
    os.makedirs(os.path.join("test", "with_numbers", "images"))
if not os.path.exists(os.path.join("test", "without_numbers", "   images")):
    os.makedirs(os.path.join("test", "without_numbers", "images"))
count = 300
for i in tqdm(range(count)):
    random_code = np.random.randint(999999999999, 9999999999999, dtype=np.int64)
    random_code = get_check_digit(random_code)
    with open("{}/with_numbers/images/{}_{}_1.jpg".format(path, str(random_code), str(i)), "wb") as file:
        EAN13(str(random_code), writer=ImageWriter()).write(file)
    # if i % 2 == 0:
    image = cv2.imread("{}/with_numbers/images/{}_{}_1.jpg".format(path, str(random_code), str(i)), cv2.IMREAD_GRAYSCALE)
    image = np.asarray(image)[2:199, 63:460]
    image = cv2.resize(image, (320, 160))
    cv2.imwrite("{}/without_numbers/images/{}_{}_0.jpg".format(path, str(random_code), str(i)), image)

In [ ]:
def filter_predicitons_tf(y_pred):
    barcode_list = []
    y_pred = np.reshape(y_pred, (-1, 13, 10))
#     print(tf.make_ndarray(y_pred))
    pred_idx = np.argmax(y_pred, axis=-1)
    for pred_barcode in pred_idx:
        barcode = ""
        for num in pred_barcode:
            barcode += str(num)
        barcode_list.append(barcode)
    return barcode_list

def humming_distance(string_1, string_2):
    i = 0
    count = 0
    while i < len(string_1):
        if string_1[i] != string_2[i]:
            count += 1
        i += 1
    return count

In [ ]:
res_count = 0
for image_name in os.listdir("/kaggle/working/test/without_numbers/images"):
    image = cv2.imread("/kaggle/working/test/without_numbers/images/"+ image_name, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT))
    image = np.expand_dims(image, -1)
    image = image / 128
    image = np.asarray([image])
    res = model.predict([image])
    res = filter_predicitons_tf(res)
#     print(image_name.split("_")[0], res[0])
    dist = humming_distance(image_name.split("_")[0], res[0])
    if dist == 0:
        res_count = res_count + 1
    print(image_name, " : ", res[0], " : ", dist)